In [1]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib

import pandas as pd
import numpy as np
import copy
import time
import re
import matplotlib.pyplot as plt

In [2]:
# 音を出す
def sound():
    !rundll32 user32.dll,MessageBeep

In [6]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 2915
# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)
sound()

# 2000ページ以降はSUUMO側が耐えないらしいよ

1ページ目：5.891139030456543秒
総取得件数：213
完了：0.034%
残り時間：4時間46分6秒

2ページ目：7.795952320098877秒
総取得件数：579
完了：0.069%
残り時間：5時間32分15秒

3ページ目：7.922363519668579秒
総取得件数：1010
完了：0.103%
残り時間：5時間49分35秒

4ページ目：5.886821269989014秒
総取得件数：1225
完了：0.137%
残り時間：5時間33分30秒

5ページ目：5.917369842529297秒
総取得件数：1392
完了：0.172%
残り時間：5時間24分6秒

6ページ目：7.69843316078186秒
総取得件数：1825
完了：0.206%
残り時間：5時間32分12秒

7ページ目：6.3804240226745605秒
総取得件数：2030
完了：0.24%
残り時間：5時間28分49秒

8ページ目：6.840943813323975秒
総取得件数：2287
完了：0.274%
残り時間：5時間29分3秒

9ページ目：6.853782892227173秒
総取得件数：2528
完了：0.309%
残り時間：5時間29分16秒

10ページ目：7.5822672843933105秒
総取得件数：2817
完了：0.343%
残り時間：5時間32分57秒

11ページ目：6.3669750690460205秒
総取得件数：3046
完了：0.377%
残り時間：5時間30分36秒

12ページ目：7.929954528808594秒
総取得件数：3290
完了：0.412%
残り時間：5時間34分55秒

13ページ目：6.377439260482788秒
総取得件数：3497
完了：0.446%
残り時間：5時間32分46秒

14ページ目：5.727058172225952秒
総取得件数：3661
完了：0.48%
残り時間：5時間28分40秒

15ページ目：5.554433107376099秒
総取得件数：3806
完了：0.515%
残り時間：5時間24分33秒

16ページ目：5.391765117645264秒
総取得件数：3918
完了：0.549%
残り時間：5時間20分27秒

17ページ目

130ページ目：6.059624910354614秒
総取得件数：25094
完了：4.46%
残り時間：5時間15分15秒

131ページ目：5.726823329925537秒
総取得件数：25218
完了：4.494%
残り時間：5時間14分45秒

132ページ目：5.956931114196777秒
総取得件数：25381
完了：4.528%
残り時間：5時間14分21秒

133ページ目：6.394580364227295秒
総取得件数：25581
完了：4.563%
残り時間：5時間14分6秒

134ページ目：11.41183853149414秒
総取得件数：25750
完了：4.597%
残り時間：5時間15分36秒

135ページ目：5.9720752239227295秒
総取得件数：25927
完了：4.631%
残り時間：5時間15分12秒

136ページ目：5.9200849533081055秒
総取得件数：26064
完了：4.666%
残り時間：5時間14分47秒

137ページ目：6.370475769042969秒
総取得件数：26247
完了：4.7%
残り時間：5時間14分31秒

138ページ目：6.2474365234375秒
総取得件数：26406
完了：4.734%
残り時間：5時間14分14秒

139ページ目：6.322934627532959秒
総取得件数：26547
完了：4.768%
残り時間：5時間13分57秒

140ページ目：6.426831007003784秒
総取得件数：26739
完了：4.803%
残り時間：5時間13分44秒

141ページ目：6.171734094619751秒
総取得件数：26911
完了：4.837%
残り時間：5時間13分25秒

142ページ目：6.689643383026123秒
総取得件数：27163
完了：4.871%
残り時間：5時間13分16秒

143ページ目：5.9872825145721436秒
総取得件数：27296
完了：4.906%
残り時間：5時間12分54秒

144ページ目：5.794324159622192秒
総取得件数：27446
完了：4.94%
残り時間：5時間12分28秒

145ページ目：7.151917219161987秒
総取

258ページ目：6.069519758224487秒
総取得件数：46039
完了：8.851%
残り時間：5時間5分26秒

259ページ目：6.155915021896362秒
総取得件数：46167
完了：8.885%
残り時間：5時間5分11秒

260ページ目：6.208731651306152秒
総取得件数：46285
完了：8.919%
残り時間：5時間4分58秒

261ページ目：6.228563547134399秒
総取得件数：46450
完了：8.954%
残り時間：5時間4分44秒

262ページ目：6.460227966308594秒
総取得件数：46603
完了：8.988%
残り時間：5時間4分33秒

263ページ目：6.503612756729126秒
総取得件数：46792
完了：9.022%
残り時間：5時間4分22秒

264ページ目：17.81917428970337秒
総取得件数：47002
完了：9.057%
残り時間：5時間6分5秒

265ページ目：6.1469855308532715秒
総取得件数：47137
完了：9.091%
残り時間：5時間5分50秒

266ページ目：8.682870626449585秒
総取得件数：47299
完了：9.125%
残り時間：5時間6分1秒

267ページ目：7.007394313812256秒
総取得件数：47422
完了：9.16%
残り時間：5時間5分54秒

268ページ目：6.366147518157959秒
総取得件数：47547
完了：9.194%
残り時間：5時間5分42秒

269ページ目：6.4040820598602295秒
総取得件数：47670
完了：9.228%
残り時間：5時間5分30秒

270ページ目：7.490839242935181秒
総取得件数：47846
完了：9.262%
残り時間：5時間5分28秒

271ページ目：6.560649633407593秒
総取得件数：48027
完了：9.297%
残り時間：5時間5分18秒

272ページ目：6.889871597290039秒
総取得件数：48230
完了：9.331%
残り時間：5時間5分11秒

273ページ目：7.156445741653442秒
総取得件数：48394
完了

385ページ目：6.416501998901367秒
総取得件数：62651
完了：13.208%
残り時間：4時間53分43秒

386ページ目：6.789044141769409秒
総取得件数：62802
完了：13.242%
残り時間：4時間53分35秒

387ページ目：10.8952796459198秒
総取得件数：62968
完了：13.276%
残り時間：4時間53分54秒

388ページ目：6.547549247741699秒
総取得件数：63121
完了：13.31%
残り時間：4時間53分44秒

389ページ目：6.6887876987457275秒
総取得件数：63226
完了：13.345%
残り時間：4時間53分35秒

390ページ目：7.065445899963379秒
総取得件数：63353
完了：13.379%
残り時間：4時間53分29秒

391ページ目：6.475948333740234秒
総取得件数：63480
完了：13.413%
残り時間：4時間53分19秒

392ページ目：6.823171854019165秒
総取得件数：63637
完了：13.448%
残り時間：4時間53分11秒

393ページ目：6.885709762573242秒
総取得件数：63741
完了：13.482%
残り時間：4時間53分3秒

394ページ目：7.953898191452026秒
総取得件数：63899
完了：13.516%
残り時間：4時間53分3秒

395ページ目：6.577033281326294秒
総取得件数：63999
完了：13.551%
残り時間：4時間52分53秒

396ページ目：13.673466444015503秒
総取得件数：64101
完了：13.585%
残り時間：4時間53分29秒

397ページ目：6.641231060028076秒
総取得件数：64228
完了：13.619%
残り時間：4時間53分19秒

398ページ目：6.28804349899292秒
総取得件数：64312
完了：13.654%
残り時間：4時間53分8秒

399ページ目：13.038431882858276秒
総取得件数：64473
完了：13.688%
残り時間：4時間53分39秒

400ページ目：7.792

510ページ目：6.831506252288818秒
総取得件数：79161
完了：17.496%
残り時間：4時間45分32秒

511ページ目：7.450793981552124秒
総取得件数：79314
完了：17.53%
残り時間：4時間45分27秒

512ページ目：6.848802804946899秒
総取得件数：79418
完了：17.564%
残り時間：4時間45分18秒

513ページ目：7.927501916885376秒
総取得件数：79524
完了：17.599%
残り時間：4時間45分15秒

514ページ目：6.98499608039856秒
総取得件数：79633
完了：17.633%
残り時間：4時間45分7秒

515ページ目：10.08029055595398秒
総取得件数：79774
完了：17.667%
残り時間：4時間45分14秒

516ページ目：7.432101249694824秒
総取得件数：79946
完了：17.702%
残り時間：4時間45分8秒

517ページ目：10.186335563659668秒
総取得件数：80069
完了：17.736%
残り時間：4時間45分15秒

518ページ目：7.7329161167144775秒
総取得件数：80260
完了：17.77%
残り時間：4時間45分11秒

519ページ目：6.7570600509643555秒
総取得件数：80365
完了：17.804%
残り時間：4時間45分2秒

520ページ目：7.334327459335327秒
総取得件数：80499
完了：17.839%
残り時間：4時間44分56秒

521ページ目：6.802199602127075秒
総取得件数：80609
完了：17.873%
残り時間：4時間44分47秒

522ページ目：6.481091499328613秒
総取得件数：80695
完了：17.907%
残り時間：4時間44分37秒

523ページ目：6.574520587921143秒
総取得件数：80780
完了：17.942%
残り時間：4時間44分27秒

524ページ目：7.4364259243011475秒
総取得件数：80902
完了：17.976%
残り時間：4時間44分21秒

525ページ目：6.87

...

2344ページ目：1.2870192527770996秒
総取得件数：223085
完了：80.412%
残り時間：1時間7分39秒

2345ページ目：1.1950790882110596秒
総取得件数：223085
完了：80.446%
残り時間：1時間7分30秒

2346ページ目：1.6177232265472412秒
総取得件数：223085
完了：80.48%
残り時間：1時間7分22秒

2347ページ目：1.2648000717163086秒
総取得件数：223085
完了：80.515%
残り時間：1時間7分13秒

2348ページ目：6.611243009567261秒
総取得件数：223085
完了：80.549%
残り時間：1時間7分6秒

2349ページ目：2.2956249713897705秒
総取得件数：223085
完了：80.583%
残り時間：1時間6分58秒

2350ページ目：4.0404112339019775秒
総取得件数：223085
完了：80.617%
残り時間：1時間6分50秒

2351ページ目：1.2796640396118164秒
総取得件数：223085
完了：80.652%
残り時間：1時間6分41秒

2352ページ目：1.238588571548462秒
総取得件数：223085
完了：80.686%
残り時間：1時間6分33秒

2353ページ目：1.2680754661560059秒
総取得件数：223085
完了：80.72%
残り時間：1時間6分24秒

2354ページ目：1.2534534931182861秒
総取得件数：223085
完了：80.755%
残り時間：1時間6分16秒

2355ページ目：1.2853741645812988秒
総取得件数：223085
完了：80.789%
残り時間：1時間6分8秒

2356ページ目：1.242640495300293秒
総取得件数：223085
完了：80.823%
残り時間：1時間5分59秒

2357ページ目：1.1619791984558105秒
総取得件数：223085
完了：80.858%
残り時間：1時間5分51秒

2358ページ目：1.2412736415863037秒
総取得件数：223085
完了：80.892%
残り

2465ページ目：1.2436842918395996秒
総取得件数：223085
完了：84.563%
残り時間：0時間51分13秒

2466ページ目：1.2783925533294678秒
総取得件数：223085
完了：84.597%
残り時間：0時間51分5秒

2467ページ目：1.2800798416137695秒
総取得件数：223085
完了：84.631%
残り時間：0時間50分58秒

2468ページ目：4.493967533111572秒
総取得件数：223085
完了：84.666%
残り時間：0時間50分50秒

2469ページ目：1.2837581634521484秒
総取得件数：223085
完了：84.7%
残り時間：0時間50分42秒

2470ページ目：1.2860188484191895秒
総取得件数：223085
完了：84.734%
残り時間：0時間50分35秒

2471ページ目：1.2934327125549316秒
総取得件数：223085
完了：84.768%
残り時間：0時間50分27秒

2472ページ目：1.2109575271606445秒
総取得件数：223085
完了：84.803%
残り時間：0時間50分19秒

2473ページ目：3.5922725200653076秒
総取得件数：223085
完了：84.837%
残り時間：0時間50分12秒

2474ページ目：1.269639253616333秒
総取得件数：223085
完了：84.871%
残り時間：0時間50分4秒

2475ページ目：1.2685489654541016秒
総取得件数：223085
完了：84.906%
残り時間：0時間49分56秒

2476ページ目：2.2342607975006104秒
総取得件数：223085
完了：84.94%
残り時間：0時間49分48秒

2477ページ目：1.2080540657043457秒
総取得件数：223085
完了：84.974%
残り時間：0時間49分41秒

2478ページ目：1.2540912628173828秒
総取得件数：223085
完了：85.009%
残り時間：0時間49分33秒

2479ページ目：1.2239866256713867秒
総取得件数：223085

2585ページ目：1.2369778156280518秒
総取得件数：223085
完了：88.679%
残り時間：0時間36分12秒

2586ページ目：1.2351431846618652秒
総取得件数：223085
完了：88.714%
残り時間：0時間36分4秒

2587ページ目：1.267061710357666秒
総取得件数：223085
完了：88.748%
残り時間：0時間35分57秒

2588ページ目：1.2832679748535156秒
総取得件数：223085
完了：88.782%
残り時間：0時間35分50秒

2589ページ目：1.23805570602417秒
総取得件数：223085
完了：88.816%
残り時間：0時間35分43秒

2590ページ目：1.3436942100524902秒
総取得件数：223085
完了：88.851%
残り時間：0時間35分35秒

2591ページ目：1.2690715789794922秒
総取得件数：223085
完了：88.885%
残り時間：0時間35分28秒

2592ページ目：1.4763600826263428秒
総取得件数：223085
完了：88.919%
残り時間：0時間35分21秒

2593ページ目：1.1427671909332275秒
総取得件数：223085
完了：88.954%
残り時間：0時間35分14秒

2594ページ目：1.1637604236602783秒
総取得件数：223085
完了：88.988%
残り時間：0時間35分6秒

2595ページ目：1.2327179908752441秒
総取得件数：223085
完了：89.022%
残り時間：0時間34分59秒

2596ページ目：2.283630132675171秒
総取得件数：223085
完了：89.057%
残り時間：0時間34分52秒

2597ページ目：1.2821495532989502秒
総取得件数：223085
完了：89.091%
残り時間：0時間34分45秒

2598ページ目：1.2348549365997314秒
総取得件数：223085
完了：89.125%
残り時間：0時間34分38秒

2599ページ目：1.2215375900268555秒
総取得件数：22308

2705ページ目：1.219153642654419秒
総取得件数：223085
完了：92.796%
残り時間：0時間22分15秒

2706ページ目：1.2817509174346924秒
総取得件数：223085
完了：92.83%
残り時間：0時間22分9秒

2707ページ目：1.270094871520996秒
総取得件数：223085
完了：92.864%
残り時間：0時間22分2秒

2708ページ目：1.2951011657714844秒
総取得件数：223085
完了：92.899%
残り時間：0時間21分55秒

2709ページ目：1.2401964664459229秒
総取得件数：223085
完了：92.933%
残り時間：0時間21分48秒

2710ページ目：2.2354140281677246秒
総取得件数：223085
完了：92.967%
残り時間：0時間21分42秒

2711ページ目：1.1745352745056152秒
総取得件数：223085
完了：93.002%
残り時間：0時間21分35秒

2712ページ目：1.2498219013214111秒
総取得件数：223085
完了：93.036%
残り時間：0時間21分28秒

2713ページ目：1.264103889465332秒
総取得件数：223085
完了：93.07%
残り時間：0時間21分22秒

2714ページ目：1.2717347145080566秒
総取得件数：223085
完了：93.105%
残り時間：0時間21分15秒

2715ページ目：1.2523868083953857秒
総取得件数：223085
完了：93.139%
残り時間：0時間21分8秒

2716ページ目：1.252448320388794秒
総取得件数：223085
完了：93.173%
残り時間：0時間21分1秒

2717ページ目：1.2795584201812744秒
総取得件数：223085
完了：93.208%
残り時間：0時間20分55秒

2718ページ目：1.2546770572662354秒
総取得件数：223085
完了：93.242%
残り時間：0時間20分48秒

2719ページ目：1.253307580947876秒
総取得件数：223085
完了：

2825ページ目：1.2865440845489502秒
総取得件数：223085
完了：96.913%
残り時間：0時間9分13秒

2826ページ目：1.262498140335083秒
総取得件数：223085
完了：96.947%
残り時間：0時間9分6秒

2827ページ目：1.2792201042175293秒
総取得件数：223085
完了：96.981%
残り時間：0時間9分0秒

2828ページ目：1.2703640460968018秒
総取得件数：223085
完了：97.015%
残り時間：0時間8分54秒

2829ページ目：1.2643918991088867秒
総取得件数：223085
完了：97.05%
残り時間：0時間8分47秒

2830ページ目：1.2493910789489746秒
総取得件数：223085
完了：97.084%
残り時間：0時間8分41秒

2831ページ目：1.2850794792175293秒
総取得件数：223085
完了：97.118%
残り時間：0時間8分35秒

2832ページ目：4.548682451248169秒
総取得件数：223085
完了：97.153%
残り時間：0時間8分29秒

2833ページ目：1.1738908290863037秒
総取得件数：223085
完了：97.187%
残り時間：0時間8分22秒

2834ページ目：1.1456868648529053秒
総取得件数：223085
完了：97.221%
残り時間：0時間8分16秒

2835ページ目：1.2827415466308594秒
総取得件数：223085
完了：97.256%
残り時間：0時間8分10秒

2836ページ目：2.273357391357422秒
総取得件数：223085
完了：97.29%
残り時間：0時間8分4秒

2837ページ目：1.2893917560577393秒
総取得件数：223085
完了：97.324%
残り時間：0時間7分57秒

2838ページ目：1.188678503036499秒
総取得件数：223085
完了：97.358%
残り時間：0時間7分51秒

2839ページ目：1.2620282173156738秒
総取得件数：223085
完了：97.393%
残り時間

In [7]:
columns = ['建物名','住所','最寄り駅1','最寄り駅2','最寄り駅3','築年数','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積','url']
suumo_tokyo = pd.DataFrame(data_samples, columns=columns)

In [23]:
suumo_tokyo

,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積,url
0,プチ・ボヌール,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,築4年,2階建,2階,14.8万円,5000円,14.8万円,14.8万円,42DK,48.87m2,https://suumo.jp/chintai/jnc_000046696863/?bc=...
1,コンフォリア町屋,東京都荒川区荒川５,東京メトロ千代田線/町屋駅 歩8分,京成本線/新三河島駅 歩6分,都電荒川線/町屋二丁目駅 歩7分,築16年,11階建,2階,12.1万円,7000円,12.1万円,-,1LDK,39m2,https://suumo.jp/chintai/jnc_000075965634/?bc=...
2,コンフォリア町屋,東京都荒川区荒川５,東京メトロ千代田線/町屋駅 歩8分,京成本線/新三河島駅 歩6分,都電荒川線/町屋二丁目駅 歩7分,築16年,11階建,5階,12.2万円,7000円,12.2万円,-,1LDK,39m2,https://suumo.jp/chintai/jnc_000077047086/?bc=...
3,ＪＲ常磐線 三河島駅 14階建 新築,東京都荒川区東日暮里６,ＪＲ常磐線/三河島駅 歩4分,ＪＲ山手線/日暮里駅 歩11分,東京メトロ千代田線/西日暮里駅 歩14分,新築,14階建,2階,9.5万円,10000円,9.5万円,-,1K,25.01m2,https://suumo.jp/chintai/jnc_000075599259/?bc=...
4,ＪＲ常磐線 三河島駅 14階建 新築,東京都荒川区東日暮里６,ＪＲ常磐線/三河島駅 歩4分,ＪＲ山手線/日暮里駅 歩11分,東京メトロ千代田線/西日暮里駅 歩14分,新築,14階建,2階,9.8万円,10000円,9.8万円,-,1DK,25.01m2,https://suumo.jp/chintai/jnc_000075599258/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,イーストン リバーサイド6,東京都北区志茂３,東京メトロ南北線/志茂駅 歩7分,埼玉高速鉄道/赤羽岩淵駅 歩21分,ＪＲ京浜東北線/赤羽駅 歩21分,築3年,3階建,3階,12.6万円,3000円,-,12.6万円,1LDK,45.8m2,https://suumo.jp/chintai/jnc_000075832934/?bc=...
256,中目黒4丁目貸家,東京都目黒区中目黒４,東急東横線/中目黒駅 歩12分,ＪＲ山手線/恵比寿駅 歩19分,ＪＲ山手線/目黒駅 歩21分,築8年,2階建,1階,26万円,-,26万円,26万円,1SLDK,58.23m2,https://suumo.jp/chintai/jnc_000026692811/?bc=...
257,中目黒4丁目貸家,東京都目黒区中目黒４,東急東横線/中目黒駅 歩12分,ＪＲ山手線/恵比寿駅 歩19分,ＪＲ山手線/目黒駅 歩21分,築8年,2階建,1-2階,26万円,-,26万円,26万円,1SLDK,58.23m2,https://suumo.jp/chintai/jnc_000028002813/?bc=...
258,ライジングプレイス月島,東京都中央区月島４,東京メトロ有楽町線/月島駅 歩2分,都営大江戸線/勝どき駅 歩9分,ＪＲ京葉線/越中島駅 歩17分,築12年,7階建,2階,10万円,10000円,-,-,1K,25.03m2,https://suumo.jp/chintai/jnc_000076772229/?bc=...


In [8]:
suumo_tokyo.to_csv(f'suumo{max_page}.csv', columns=columns, index=False)

In [9]:
suumo_tokyo = pd.read_csv(f'suumo{max_page}.csv')

In [12]:
suumo_tokyo

,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積,url
0,プチ・ボヌール,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,築4年,2階建,2階,14.8万円,5000円,14.8万円,14.8万円,42DK,48.87m2,https://suumo.jp/chintai/jnc_000046696863/?bc=...
1,シティインデックス西日暮里,東京都荒川区西日暮里６,ＪＲ山手線/西日暮里駅 歩7分,東京メトロ千代田線/西日暮里駅 歩6分,京成本線/新三河島駅 歩8分,築11年,14階建,7階,12.5万円,10000円,-,-,1LDK,32.08m2,https://suumo.jp/chintai/jnc_000056910077/?bc=...
2,OLD PAL,東京都荒川区東尾久４,ＪＲ山手線/田端駅 歩12分,日暮里・舎人ライナー/赤土小学校前駅 歩3分,都電荒川線/東尾久三丁目駅 歩10分,築17年,6階建,3階,9.8万円,8000円,9.8万円,9.8万円,1DK,32.2m2,https://suumo.jp/chintai/jnc_000077228173/?bc=...
3,OLD PAL,東京都荒川区東尾久４,ＪＲ山手線/田端駅 歩12分,日暮里・舎人ライナー/赤土小学校前駅 歩3分,都電荒川線/東尾久三丁目駅 歩10分,築17年,6階建,3階,15.4万円,10000円,15.4万円,15.4万円,2LDK,60.5m2,https://suumo.jp/chintai/jnc_000076950589/?bc=...
4,OLD PAL,東京都荒川区東尾久４,ＪＲ山手線/田端駅 歩12分,日暮里・舎人ライナー/赤土小学校前駅 歩3分,都電荒川線/東尾久三丁目駅 歩10分,築17年,6階建,5階,15.7万円,10000円,15.7万円,15.7万円,2LDK,59.6m2,https://suumo.jp/chintai/jnc_000077228174/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223080,メゾンルミエル,東京都目黒区下目黒４,東急目黒線/不動前駅 歩12分,ＪＲ山手線/目黒駅 歩16分,東急目黒線/武蔵小山駅 歩16分,築31年,2階建,2階,8.5万円,-,8.5万円,8.5万円,1K,27.6m2,https://suumo.jp/chintai/jnc_000064271776/?bc=...
223081,メゾンルミエル,東京都目黒区下目黒４,東急目黒線/不動前駅 歩12分,ＪＲ山手線/目黒駅 歩16分,東急目黒線/武蔵小山駅 歩16分,築31年,2階建,2階,8.7万円,-,8.7万円,8.7万円,1K,27.6m2,https://suumo.jp/chintai/jnc_000065859312/?bc=...
223082,メゾンルミエル,東京都目黒区下目黒４,東急目黒線/不動前駅 歩12分,ＪＲ山手線/目黒駅 歩16分,東急目黒線/武蔵小山駅 歩16分,築31年,2階建,2階,8.9万円,-,8.9万円,8.9万円,1K,27.6m2,https://suumo.jp/chintai/jnc_000062882505/?bc=...
223083,東急目黒線 不動前駅 2階建 築5年,東京都品川区西五反田４,東急目黒線/不動前駅 歩6分,東急目黒線/武蔵小山駅 歩13分,ＪＲ山手線/目黒駅 歩20分,築5年,2階建,1階,6.8万円,3500円,-,-,ワンルーム,10.03m2,https://suumo.jp/chintai/jnc_000070439141/?bc=...


In [14]:
# 管理費（正規表現）
suumo_tokyo['管理費']

# 管理費の'-'を0に置き換える
suumo_tokyo.loc[suumo_tokyo['管理費']=="-",'管理費'] = suumo_tokyo.loc[suumo_tokyo['管理費']=="-",'管理費'].replace('-','0')

# 数字+文字→数字
def numobj_num(x):
    return re.sub(r'([0-9]+)\w+',r'\1',x)

# ????円→????
suumo_tokyo['管理費'] = suumo_tokyo['管理費'].map(lambda x: numobj_num(x))
# 整数型に変更
suumo_tokyo['管理費'] = suumo_tokyo['管理費'].astype(int)

In [15]:
# 築年数（正規表現）

# 新築は築0年ということにする
suumo_tokyo.loc[suumo_tokyo['築年数']=='新築','築年数'] = '築0年'

# 文字+数字+文字→数字
def objnumobj_num(x):
    return re.sub(r'\w+?([0-9]+)\w+',r'\1',x)

# 築??年(以上)→??
# 99階以上は99階に圧縮されます
suumo_tokyo['築年数'] = suumo_tokyo['築年数'] = suumo_tokyo['築年数'].map(lambda x: objnumobj_num(x))

In [16]:
# 地下変数（正規表現）

# 平屋は1階建ということにする
suumo_tokyo.loc[suumo_tokyo['階数']=='平屋','階数'] = '1階建'

# この文字列ありますか？関数
def search_object(search,object):
    return bool(re.search(search,object))

# 地下何階までありますか？関数
def underground(x):
    return search_object('地下',x)*objnumobj_num(x)+(1-search_object('地下',x))*'0'
    
# 地下がなければ０、あるなら何階まであるか
suumo_tokyo['地下'] = suumo_tokyo['階数'].map(lambda x: underground(x))

# int型に変換
suumo_tokyo['地下'] = suumo_tokyo['地下'].astype('int')

In [17]:
# 地上変数(正規表現)
suumo_tokyo['階数']

# 平屋は1階建ということにする
suumo_tokyo.loc[suumo_tokyo['階数']=='平屋','階数'] = '1階建'

# 最後の数字列を取り出す
def lastnum(x):
    return re.findall('[0-9]+',x)[-1]

# ??階建→??
suumo_tokyo['地上'] = suumo_tokyo['階数'].map(lambda x: lastnum(x))

# int型に変換
suumo_tokyo['地上'] = suumo_tokyo['地上'].astype(int)

In [18]:
# 階数変数は地下地上の合算にしておく
suumo_tokyo['階数'] = suumo_tokyo['地上'] + suumo_tokyo['地下']

In [19]:
# 階変数(正規表現)

# - は1階ってことにする
suumo_tokyo.loc[suumo_tokyo['階']=='-','階'] = '1階'

# (文字)数字を取り出す
def get_objnum(x):
    return re.search(r'([A-Z]?)([0-9]+)',x).group()

# 階の前処理 -があればその後ろ、なければ(文字)数字
def floor(x):
    return search_object('-',x)*lastnum(x)+(1-search_object('-',x))*get_objnum(x)
# B? → -?,Bがなければそのまま
def basement_floor(x):
    return search_object('B',x)*('-'+re.search(r'[0-9]+',x).group())+(1-search_object('B',x))*(x)
# M? → ?.5 Mがなければそのまま
def middle_floor(x):
    return search_object('M',x)*(re.search(r'[0-9]+',x).group()+'.5')+(1-search_object('M',x))*(x)
   
# 上の処理を適用する
suumo_tokyo['階'] = suumo_tokyo['階'].map(lambda x: floor(x)).map(lambda x: basement_floor(x)).map(lambda x: middle_floor(x))

# float型に変換
suumo_tokyo['階'] = suumo_tokyo['階'].astype(float)

In [20]:
# 賃料変数(正規表現)

# ???万円→???
def drop_man(x):
    return re.sub(r'([0-9]+)万円',r'\1',x)

# 賃料から万円を除去
suumo_tokyo['賃料'] = suumo_tokyo['賃料'].map(lambda x: drop_man(x))
# float型に変換
suumo_tokyo['賃料'] = suumo_tokyo['賃料'].astype(float)

In [21]:
# 礼金変数(正規表現)

# '-'は0万円ってことにする
suumo_tokyo.loc[suumo_tokyo['敷金']=='-','敷金'] = '0万円'

# 敷金から万円を除去
suumo_tokyo['敷金'] = suumo_tokyo['敷金'].map(lambda x: drop_man(x))
# float型に変換
suumo_tokyo['敷金'] = suumo_tokyo['敷金'].astype(float)

In [22]:
# 礼金変数(正規表現)

# '-'は0万円ってことにする
suumo_tokyo.loc[suumo_tokyo['礼金']=='-','礼金'] = '0万円'

# 礼金から万円を除去
suumo_tokyo['礼金'] = suumo_tokyo['礼金'].map(lambda x: drop_man(x))
# float型に変換
suumo_tokyo['礼金'] = suumo_tokyo['礼金'].astype(float)

In [ ]:
# 間取り変換案
# そのまま
# 部屋数、onehot(L,D,K)に変換

In [23]:
# 専有面積変数（正規表現）

# 英字以降を切り落とす
def drop_behind_alfa(x):
    return re.sub('[a-z]\w+','',x)
# ????m2 → ????
suumo_tokyo['専有面積'] = suumo_tokyo['専有面積'].map(lambda x: drop_behind_alfa(x))
# float型に変換
suumo_tokyo['専有面積'] = suumo_tokyo['専有面積'].astype(float)

In [24]:
# 住所変数（正規表現）

# ～都,～区,～ に分けたい
# ついでに[都道府県][市区町村][以下]に対応したい

x = suumo_tokyo.loc[0,'住所']

# 住所を(都道府県)(市区町村)(市区町村以下)に分けて取り出す
def split_address(x):
    a,b,c = re.search('(...??[都道府県])(.+?[市区町村])(.+)',x).groups()
    return a,b,c

# 一応分割できたっぽい
suumo_tokyo[['都道府県','市区町村','市区町村以下']] = suumo_tokyo.apply(lambda x:split_address(x.住所),axis=1,result_type='expand')

In [140]:
# 最寄り駅変数（正規表現）

# 最寄り駅から路線と駅を取り出す
suumo_tokyo[['路線','駅']] = suumo_tokyo.apply(lambda x:re.search('(.+)/(.+?)\s(.+)',x.最寄り駅1).groups()[0:2],axis=1,result_type='expand')

def how_to_station():
    # 移動手段は3パターン
    how_to_station = ['歩','バス','車']
    for by in how_to_station:
        # 移動手段変数を生成
        suumo_tokyo[f'{by}'] = np.zeros(suumo_tokyo.shape[0])
        # その移動手段があるなら移動手段変数に格納する
        suumo_tokyo.loc[suumo_tokyo['最寄り駅1'].map(lambda x: search_object(f'{by}[0-9]+分',x)),f'{by}'] = suumo_tokyo.loc[suumo_tokyo['最寄り駅1'].map(lambda x: search_object(f'{by}[0-9]+分',x)),'最寄り駅1'].map(lambda x: re.search(f'{by}([0-9]+)分',x).group(1))
        # 所要時間をint型に変換
        suumo_tokyo[f'{by}'] = suumo_tokyo[f'{by}'].astype(int)

# 移動手段ごとに所用時間変数を作る
how_to_station()

In [173]:
# 順番を見やすいように入れ替える
columns = ['建物名','都道府県','市区町村','市区町村以下','路線','駅','歩','バス','車','築年数','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積','url']
suumo_tokyo_shaped = suumo_tokyo[columns]
# 重複を削除する（重複10000くらいありましたね…）
suumo_tokyo_shaped = suumo_tokyo_shaped[suumo_tokyo_shaped.duplicated() == False]
# 保存
suumo_tokyo_shaped.to_csv('SuumoTokyoShaped20221006.csv', index=False)

In [174]:
pd.read_csv('SuumoTokyoShaped20221006.csv')

,建物名,都道府県,市区町村,市区町村以下,路線,駅,歩,バス,車,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積,url
0,プチ・ボヌール,東京都,大田区,南蒲田３,京急空港線,糀谷駅,7,0,0,4,2,2.0,14.8,5000,14.8,14.8,42DK,48.87,https://suumo.jp/chintai/jnc_000046696863/?bc=...
1,シティインデックス西日暮里,東京都,荒川区,西日暮里６,ＪＲ山手線,西日暮里駅,7,0,0,11,14,7.0,12.5,10000,0.0,0.0,1LDK,32.08,https://suumo.jp/chintai/jnc_000056910077/?bc=...
2,OLD PAL,東京都,荒川区,東尾久４,ＪＲ山手線,田端駅,12,0,0,17,6,3.0,9.8,8000,9.8,9.8,1DK,32.20,https://suumo.jp/chintai/jnc_000077228173/?bc=...
3,OLD PAL,東京都,荒川区,東尾久４,ＪＲ山手線,田端駅,12,0,0,17,6,3.0,15.4,10000,15.4,15.4,2LDK,60.50,https://suumo.jp/chintai/jnc_000076950589/?bc=...
4,OLD PAL,東京都,荒川区,東尾久４,ＪＲ山手線,田端駅,12,0,0,17,6,5.0,15.7,10000,15.7,15.7,2LDK,59.60,https://suumo.jp/chintai/jnc_000077228174/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211279,ヴェール 白鷺 Ｂ棟,東京都,中野区,白鷺３,西武新宿線,鷺ノ宮駅,3,0,0,26,2,1.0,6.0,0,12.0,0.0,ワンルーム,19.02,https://suumo.jp/chintai/jnc_000013450339/?bc=...
211280,ヴェール 白鷺 Ｂ棟,東京都,中野区,白鷺３,西武新宿線,鷺ノ宮駅,3,0,0,26,2,2.0,6.3,0,12.6,0.0,1K,20.28,https://suumo.jp/chintai/jnc_000038197370/?bc=...
211281,ＪＲ総武線 亀戸駅 4階建 築8年,東京都,江東区,亀戸２,ＪＲ総武線,亀戸駅,4,0,0,8,4,4.0,8.1,4000,8.1,0.0,ワンルーム,25.09,https://suumo.jp/chintai/jnc_000076844287/?bc=...
211282,東急目黒線 不動前駅 2階建 築5年,東京都,品川区,西五反田４,東急目黒線,不動前駅,6,0,0,5,2,1.0,6.8,3500,0.0,0.0,ワンルーム,10.03,https://suumo.jp/chintai/jnc_000070439141/?bc=...
